In [143]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser

In [144]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [145]:
# Define database and collection
db = client.nasa_db
collection = db.items

In [146]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
#print(soup.prettify())

In [147]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
#results = soup.find_all('div', class_='image_and_description_container')
results = soup.find_all('div', class_='slide')

# Print formatted version of the soup
print(results)

[<div class="slide">
<div class="image_and_description_container">
<a href="/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/">
<div class="rollover_description">
<div class="rollover_description_inner">
Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.
</div>
<div class="overlay_arrow">
<img alt="More" src="/assets/overlay-arrow.png"/>
</div>
</div>
<img alt="Alabama High School Student Names NASA's Mars Helicopter" class="img-lazy" data-lazy="/system/news_items/list_view_images/8659_1-PIA23883-MAIN-320x240.jpg" src="/assets/loading_320x240.png"/>
</a>
</div>
<div class="content_title">
<a href="/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/">
Alabama High School Student Names NASA's Mars Helicopter
</a>
</div>
</div>, <div class="slide">
<div class="image_and_description_container">
<a href="/news/8645/mars-helicopter-attached-to-nasas-perseverance-ro

In [148]:
# Testing the results in looking at the class
# results are returned as an iterable list
#results = soup.find_all('div', class_='image_and_description_container')
#results_test = soup.find_all('div', class_='content_title')

# Print formatted version of the soup
#print(results_test)

In [149]:




# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('div', class_='content_title').text

        # Identify and return the description
        article = result.find('div', class_='rollover_description_inner').text


        # Run only if title, price, and link are available
        if (title and article):
        #if (article):
            # Print results
            print('-------------')
            print(title)
            print(article)


            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'article': article
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)

-------------


Alabama High School Student Names NASA's Mars Helicopter



Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.

-------------


Mars Helicopter Attached to NASA's Perseverance Rover



The team also fueled the rover's sky crane to get ready for this summer's history-making launch.

-------------


NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes



After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.

-------------


10.9 Million Names Now Aboard NASA's Perseverance Mars Rover



As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars.

-------------


Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover



NASA chose a seventh-grader f

In [150]:
# Display items in MongoDB collection
news_listings = db.items.find()

for listing in news_listings:
    print(listing)

{'_id': ObjectId('5ed5bbb9719cec84b49bd75e'), 'title': "\n\nAlabama High School Student Names NASA's Mars Helicopter\n\n", 'article': "\nVaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.\n"}
{'_id': ObjectId('5ed5bbb9719cec84b49bd75f'), 'title': "\n\nMars Helicopter Attached to NASA's Perseverance Rover\n\n", 'article': "\nThe team also fueled the rover's sky crane to get ready for this summer's history-making launch.\n"}
{'_id': ObjectId('5ed5bbb9719cec84b49bd760'), 'title': "\n\nNASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes\n\n", 'article': '\nAfter the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.\n'}
{'_id': ObjectId('5ed5bbb9719cec84b49bd761'), 'title': "\n\n10.9 Million Names Now Aboard NASA's Perseverance Mars Rover\n\n", 'article': "\nAs part of NASA's 'Send Your Name to Mars' 

In [151]:
#try:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#except Exception as e:
#    print(e)

In [179]:
#featured_image_url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [180]:
# Iterate through all pages
#for x in range(50):
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup_mars = bs(html, 'html.parser')
#print(soup_mars)
# Retrieve all elements that contain book information
articles = soup_mars.find_all('ul', class_='articles')
print(articles)    

[<ul class="articles">
<li class="slide">
<a class="fancybox" data-description="This image from NASAs Mars Odyssey shows part of the summit caldera on Arsia Mons." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/largesize/PIA23932_hires.jpg" data-link="/spaceimages/details.php?id=PIA23932" data-thumbnail="/spaceimages/images/wallpaper/PIA23932-640x350.jpg" data-title="Arsia Mons Summit">
<div class="image_and_description_container">
<div class="rollover_description">
<h3 class="release_date">May 26, 2020</h3>
<div class="item_tease_overlay">Arsia Mons Summit</div>
<div class="overlay_arrow">
<img alt="more arrow" src="/assets/images/overlay-arrow.png"/>
</div>
</div>
<div class="img">
<img alt="Arsia Mons Summit" class="thumb" src="/spaceimages/images/wallpaper/PIA23932-640x350.jpg" title="Arsia Mons Summit"/>
</div>
<div class="list_text_content">
<div class="article_teaser_body">May 26, 2020</div>
<div class="content_title">
										  Arsia Mons Summit
								

In [181]:
url_link = []
image_list = []

# Iterate through each book
for article in articles:
    # Use Beautiful Soup's find() method to navigate and retrieve attributes
    line = article.find('li')
    link = line.find('a')
    href = link['data-fancybox-href']
#        title = link['title']
    print('-----------')
    print(href)
    print('https://www.jpl.nasa.gov' + href)
    url_link.append(href)
    image_list = ['http://books.toscrape.com/' + url for url in url_link]

# Click the 'Next' button on each page
try:
    browser.click_link_by_partial_text('next')

except:
    print("Scraping Complete")

-----------
/spaceimages/images/largesize/PIA23932_hires.jpg
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23932_hires.jpg
Scraping Complete


In [182]:
image_list

['http://books.toscrape.com//spaceimages/images/largesize/PIA23932_hires.jpg']

In [ ]:
#https://twitter.com/marswxreport?lang=en

In [188]:
#featured_image_url
url_twitter = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url_twitter)

In [191]:
# Iterate through all pages
#for x in range(50):
# HTML object
html_twitter = browser.html
# Parse HTML with Beautiful Soup
soup_twitter = bs(html_twitter, 'html.parser')
#print(soup_mars)
# Retrieve all elements that contain book information
tweets = soup_twitter.find_all('span', class_='css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0')
print(tweets)   

[<span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Log in</span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Sign up</span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">See new Tweets</span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Follow</span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Mars Weather</span></span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Mars Weather</span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">@MarsWxReport</span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Updates as avail from the REMS weather instrument aboard </span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">.  Data credit: Centro deAstrobiologia, FMI, JPL/NASA

In [215]:
    for tweet in tweets:
#        msg=(tweet.text)
#            print(msg)
#        try:
#        if 

#            print(tweet.text.is_text_present('InSight'))
            print('page:', x, '-------------')
#            print((tweet.text).startswith('InSight'))
            print(tweet.text)
#        except tweet.text = '@MarsWxReport':
#            print('NO')
    browser.click_link_by_partial_text('Next')

page: 49 -------------
Log in
page: 49 -------------
Sign up
page: 49 -------------
See new Tweets
page: 49 -------------
Follow
page: 49 -------------
Mars Weather
page: 49 -------------
Mars Weather
page: 49 -------------
@MarsWxReport
page: 49 -------------
Updates as avail from the REMS weather instrument aboard 
page: 49 -------------
.  Data credit: Centro deAstrobiologia, FMI, JPL/NASA, Not an official acct.
page: 49 -------------
Gale Crater, Mars
page: 49 -------------
Gale Crater, Mars
page: 49 -------------
51
page: 49 -------------
Following
page: 49 -------------
57.3K
page: 49 -------------
Followers
page: 49 -------------
Tweets
page: 49 -------------
Tweets & replies
page: 49 -------------
Media
page: 49 -------------
Likes
page: 49 -------------
Tweets
page: 49 -------------
Tweets & replies
page: 49 -------------
Media
page: 49 -------------
Likes
page: 49 -------------
Mars Weather
page: 49 -------------
Mars Weather
page: 49 -------------
@MarsWxReport
page: 49 ----

ElementDoesNotExist: no elements could be found with link by partial text "Next"

In [216]:
import pandas as pd
import pprint

In [217]:
url_facts = 'https://space-facts.com/mars/'

In [219]:
tables = pd.read_html(url_facts)
pprint.pprint(tables)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:    -153 to 20 °C      -88 to 58°C,
                       0  

In [220]:
type(tables)

list

In [228]:
df = tables[1]
#df.columns = ['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons', 
#              'Orbit Distance', 'Orbit Period', 'Surface Temperature', 'First Record', 
#              'Recorded By']

df.columns = ['Comparision', 'Mars', 'Earth']

df.head()

,Comparision,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
